In [ ]:
import cv2
from datetime import datetime

import pandas as pd
columns = ['preReadTime', 'postReadTime']
metaDf = pd.DataFrame(columns=columns)

# Initialize the webcam (0 is the default camera)
timeBeforeCapDefined = datetime.now() 
cap = cv2.VideoCapture(0)
timeAfterCapDefined = datetime.now() 

cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

initalFrameReadStart = datetime.now()
ret, frame = cap.read()
initalFrameReadEnd = datetime.now()
